## Bayesian methods of hyperparameter optimization

In addition to the random search and the grid search methods for selecting optimal hyperparameters, we can use Bayesian methods of probabilities to select the optimal hyperparameters for an algorithm.

In this case study, we will be using the BayesianOptimization library to perform hyperparmater tuning. This library has very good documentation which you can find here: https://github.com/fmfn/BayesianOptimization

You will need to install the Bayesian optimization module. Running a cell with an exclamation point in the beginning of the command will run it as a shell command — please do this to install this module from our notebook in the cell below.

In [1]:
#! pip install bayesian-optimization

In [2]:
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import LabelEncoder
import numpy as np
import pandas as pd
import lightgbm
from bayes_opt import BayesianOptimization
from catboost import CatBoostClassifier, cv, Pool

In [3]:
import os
os.listdir()

['.ipynb_checkpoints',
 'Bayesian_optimization_case_study.ipynb',
 'flight_delays_test.csv',
 'flight_delays_test.csv.zip',
 'flight_delays_train.csv',
 'flight_delays_train.csv.zip']

## How does Bayesian optimization work?

Bayesian optimization works by constructing a posterior distribution of functions (Gaussian process) that best describes the function you want to optimize. As the number of observations grows, the posterior distribution improves, and the algorithm becomes more certain of which regions in parameter space are worth exploring and which are not, as seen in the picture below.

<img src="https://github.com/fmfn/BayesianOptimization/blob/master/examples/bo_example.png?raw=true" />
As you iterate over and over, the algorithm balances its needs of exploration and exploitation while taking into account what it knows about the target function. At each step, a Gaussian Process is fitted to the known samples (points previously explored), and the posterior distribution, combined with an exploration strategy (such as UCB — aka Upper Confidence Bound), or EI (Expected Improvement). This process is used to determine the next point that should be explored (see the gif below).
<img src="https://github.com/fmfn/BayesianOptimization/raw/master/examples/bayesian_optimization.gif" />

## Let's look at a simple example

The first step is to create an optimizer. It uses two items:
* function to optimize
* bounds of parameters

The function is the procedure that counts metrics of our model quality. The important thing is that our optimization will maximize the value on function. Smaller metrics are best. Hint: don't forget to use negative metric values.

Here we define our simple function we want to optimize.

In [4]:
def simple_func(a, b):
    return a + b

Now, we define our bounds of the parameters to optimize, within the Bayesian optimizer.

In [5]:
optimizer = BayesianOptimization(
    simple_func,
    {'a': (1, 3),
    'b': (4, 7)})

These are the main parameters of this function:

* **n_iter:** This is how many steps of Bayesian optimization you want to perform. The more steps, the more likely you are to find a good maximum.

* **init_points:** This is how many steps of random exploration you want to perform. Random exploration can help by diversifying the exploration space.

Let's run an example where we use the optimizer to find the best values to maximize the target value for a and b given the inputs of 3 and 2.

In [6]:
optimizer.maximize(3,2)

|   iter    |  target   |     a     |     b     |
-------------------------------------------------
| 1         | 7.158     | 2.603     | 4.555     |
| 2         | 7.036     | 1.925     | 5.111     |
| 3         | 7.779     | 1.333     | 6.446     |
| 4         | 8.083     | 1.085     | 6.997     |
| 5         | 10.0      | 3.0       | 7.0       |


Great, now let's print the best parameters and the associated maximized target.

In [7]:
print(optimizer.max['params']);optimizer.max['target']

{'a': 3.0, 'b': 7.0}


10.0

## Test it on real data using the Light GBM

The dataset we will be working with is the famous flight departures dataset. Our modeling goal will be to predict if a flight departure is going to be delayed by 15 minutes based on the other attributes in our dataset. As part of this modeling exercise, we will use Bayesian hyperparameter optimization to identify the best parameters for our model.

**<font color='teal'> You can load the zipped csv files just as you would regular csv files using Pandas read_csv. In the next cell load the train and test data into two seperate dataframes. </font>**


In [8]:
train_df = pd.read_csv('flight_delays_train.csv.zip')
test_df = pd.read_csv('flight_delays_test.csv.zip')

**<font color='teal'> Print the top five rows of the train dataframe and review the columns in the data. </font>**

In [9]:
train_df.head()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance,dep_delayed_15min
0,c-8,c-21,c-7,1934,AA,ATL,DFW,732,N
1,c-4,c-20,c-3,1548,US,PIT,MCO,834,N
2,c-9,c-2,c-5,1422,XE,RDU,CLE,416,N
3,c-11,c-25,c-6,1015,OO,DEN,MEM,872,N
4,c-10,c-7,c-6,1828,WN,MDW,OMA,423,Y


**<font color='teal'> Use the describe function to review the numeric columns in the train dataframe. </font>**

In [10]:
train_df.describe()

,DepTime,Distance
count,100000.000000,100000.00000
mean,1341.523880,729.39716
std,476.378445,574.61686
min,1.000000,30.00000
25%,931.000000,317.00000
50%,1330.000000,575.00000
75%,1733.000000,957.00000
max,2534.000000,4962.00000


Notice, `DepTime` is the departure time in a numeric representation in 2400 hours. 

 **<font color='teal'>The response variable is 'dep_delayed_15min' which is a categorical column, so we need to map the Y for yes and N for no values to 1 and 0. Run the code in the next cell to do this.</font>**

In [11]:
#train_df = train_df[train_df.DepTime <= 2400].copy()
y_train = train_df['dep_delayed_15min'].map({'Y': 1, 'N': 0}).values

## Feature Engineering
Use these defined functions to create additional features for the model. Run the cell to add the functions to your workspace.

In [12]:
def label_enc(df_column):
    df_column = LabelEncoder().fit_transform(df_column)
    return df_column

def make_harmonic_features_sin(value, period=2400):
    value *= 2 * np.pi / period 
    return np.sin(value)

def make_harmonic_features_cos(value, period=2400):
    value *= 2 * np.pi / period 
    return np.cos(value)

def feature_eng(df):
    df['flight'] = df['Origin']+df['Dest']
    df['Month'] = df.Month.map(lambda x: x.split('-')[-1]).astype('int32')
    df['DayofMonth'] = df.DayofMonth.map(lambda x: x.split('-')[-1]).astype('uint8')
    df['begin_of_month'] = (df['DayofMonth'] < 10).astype('uint8')
    df['midddle_of_month'] = ((df['DayofMonth'] >= 10)&(df['DayofMonth'] < 20)).astype('uint8')
    df['end_of_month'] = (df['DayofMonth'] >= 20).astype('uint8')
    df['DayOfWeek'] = df.DayOfWeek.map(lambda x: x.split('-')[-1]).astype('uint8')
    df['hour'] = df.DepTime.map(lambda x: x/100).astype('int32')
    df['morning'] = df['hour'].map(lambda x: 1 if (x <= 11)& (x >= 7) else 0).astype('uint8')
    df['day'] = df['hour'].map(lambda x: 1 if (x >= 12) & (x <= 18) else 0).astype('uint8')
    df['evening'] = df['hour'].map(lambda x: 1 if (x >= 19) & (x <= 23) else 0).astype('uint8')
    df['night'] = df['hour'].map(lambda x: 1 if (x >= 0) & (x <= 6) else 0).astype('int32')
    df['winter'] = df['Month'].map(lambda x: x in [12, 1, 2]).astype('int32')
    df['spring'] = df['Month'].map(lambda x: x in [3, 4, 5]).astype('int32')
    df['summer'] = df['Month'].map(lambda x: x in [6, 7, 8]).astype('int32')
    df['autumn'] = df['Month'].map(lambda x: x in [9, 10, 11]).astype('int32')
    df['holiday'] = (df['DayOfWeek'] >= 5).astype(int) 
    df['weekday'] = (df['DayOfWeek'] < 5).astype(int)
    df['airport_dest_per_month'] = df.groupby(['Dest', 'Month'])['Dest'].transform('count')
    df['airport_origin_per_month'] = df.groupby(['Origin', 'Month'])['Origin'].transform('count')
    df['airport_dest_count'] = df.groupby(['Dest'])['Dest'].transform('count')
    df['airport_origin_count'] = df.groupby(['Origin'])['Origin'].transform('count')
    df['carrier_count'] = df.groupby(['UniqueCarrier'])['Dest'].transform('count')
    df['carrier_count_per month'] = df.groupby(['UniqueCarrier', 'Month'])['Dest'].transform('count')
    df['deptime_cos'] = df['DepTime'].map(make_harmonic_features_cos)
    df['deptime_sin'] = df['DepTime'].map(make_harmonic_features_sin)
    df['flightUC'] = df['flight']+df['UniqueCarrier']
    df['DestUC'] = df['Dest']+df['UniqueCarrier']
    df['OriginUC'] = df['Origin']+df['UniqueCarrier']
    return df.drop('DepTime', axis=1)

Concatenate the training and testing dataframes.


In [13]:
full_df = pd.concat([train_df.drop('dep_delayed_15min', axis=1), test_df])
full_df = feature_eng(full_df)

Apply the earlier defined feature engineering functions to the full dataframe.

In [14]:
for column in ['UniqueCarrier', 'Origin', 'Dest','flight',  'flightUC', 'DestUC', 'OriginUC']:
    full_df[column] = label_enc(full_df[column])


Split the new full dataframe into X_train and X_test. 

In [15]:
X_train = full_df[:train_df.shape[0]]
X_test = full_df[train_df.shape[0]:]

Create a list of the categorical features.

In [16]:
categorical_features = ['Month',  'DayOfWeek', 'UniqueCarrier', 'Origin', 'Dest','flight',  'flightUC', 'DestUC', 'OriginUC']

Let's build a light GBM model to test the bayesian optimizer.

### [LightGBM](https://lightgbm.readthedocs.io/en/latest/) is a gradient boosting framework that uses tree-based learning algorithms. It is designed to be distributed and efficient with the following advantages:

* Faster training speed and higher efficiency.
* Lower memory usage.
* Better accuracy.
* Support of parallel and GPU learning.
* Capable of handling large-scale data.

First, we define the function we want to maximize and that will count cross-validation metrics of lightGBM for our parameters.

Some params such as num_leaves, max_depth, min_child_samples, min_data_in_leaf should be integers.

In [17]:
def lgb_eval(num_leaves, max_depth, lambda_l2, lambda_l1, min_child_samples, min_data_in_leaf):
    params = {
        "objective": "binary",
        "metric": "auc", 
        'is_unbalance': True,
        "num_leaves": int(num_leaves),
        "max_depth": int(max_depth),
        "lambda_l2": lambda_l2,
        "lambda_l1": lambda_l1,
        "num_threads": 20,
        "min_child_samples": int(min_child_samples),
        'min_data_in_leaf': int(min_data_in_leaf),
        "learning_rate": 0.03,
        "subsample_freq": 5,
        "bagging_seed": 42,
        "verbosity": -1,
        "early_stopping_rounds": 100,
        "verbose": -1  # Add verbose parameter here
    }
    
    lgtrain = lightgbm.Dataset(X_train, y_train, categorical_feature=categorical_features)
    cv_result = lightgbm.cv(params,
                            lgtrain,
                            num_boost_round=1000,
                            stratified=True,
                            nfold=3)
    
    # Print the cv_result to inspect available metrics
    print(cv_result)
    
    # Return a placeholder value for now
    return 0


Apply the Bayesian optimizer to the function we created in the previous step to identify the best hyperparameters. We will run 10 iterations and set init_points = 2.


In [18]:
lgbBO = BayesianOptimization(lgb_eval, {'num_leaves': (25, 4000),
                                                'max_depth': (5, 63),
                                                'lambda_l2': (0.0, 0.05),
                                                'lambda_l1': (0.0, 0.05),
                                                'min_child_samples': (50, 10000),
                                                'min_data_in_leaf': (100, 2000)
                                                })

lgbBO.maximize(n_iter=10, init_points=2)

|   iter    |  target   | lambda_l1 | lambda_l2 | max_depth | min_ch... | min_da... | num_le... |
-------------------------------------------------------------------------------------------------
{'valid auc-mean': [0.6990959156076887, 0.7059307510779691, 0.7100726021331635, 0.7119904391999975, 0.7137135460584195, 0.7149647265907223, 0.7159323448328708, 0.717113936165474, 0.7176882512248396, 0.7186431005631205, 0.7193381511002673, 0.7197216989644918, 0.7198108395084134, 0.7201214020564045, 0.7202817348657028, 0.720266649050385, 0.7204859033485024, 0.7205231706764658, 0.7206477089209438, 0.7208859163011576, 0.7209092669604314, 0.7210873301348578, 0.7210842943242274, 0.7212009173890171, 0.721298543016791, 0.7214714905055549, 0.7217633227571759, 0.7219244922255803, 0.7221214510901337, 0.7222174537569216, 0.7223828576120656, 0.7224343458107105, 0.7224408311134881, 0.7224632088955248, 0.722516629907786, 0.7224714432341552, 0.7225700587671898, 0.722650637611483, 0.7227255022583874, 0.7227578

{'valid auc-mean': [0.6989784590962215, 0.7060171054613124, 0.7086452198617358, 0.7109847638602084, 0.713172681546177, 0.7141428845602352, 0.7153403770335514, 0.7161458294837398, 0.7165599024767193, 0.7172274670554857, 0.7174064415503546, 0.7177831604335801, 0.7180667044413637, 0.7183956862965711, 0.7190239695381772, 0.7193591210182134, 0.7194626573458193, 0.7197767572079359, 0.7200082424189113, 0.7202098335992085, 0.7204408913074479, 0.7208637302314935, 0.7209413229387307, 0.7211794885860005, 0.7213226130271795, 0.7214329200943519, 0.7214613833889936, 0.7214764000827113, 0.7215380139377845, 0.7216938938930314, 0.7217446490255929, 0.7218749685525978, 0.7219276140875155, 0.721999842132487, 0.7221262088233754, 0.7222558046011724, 0.7222865441490117, 0.7222953599750586, 0.7224013600973525, 0.7224761411796897, 0.7225662844033781, 0.722694276541017, 0.7227720562608716, 0.7228605695178544, 0.7229995354178529, 0.7231419220541829, 0.7232117988811915, 0.7232559479839309, 0.7232876168627381, 0.7

{'valid auc-mean': [0.6973088979513862, 0.7042473880248754, 0.7075668953127113, 0.7094499042106451, 0.7112881874066317, 0.7121425802503687, 0.7131011928542402, 0.7133537028040271, 0.7141583738033367, 0.7148075392999438, 0.7152505689760016, 0.7158325218948395, 0.7164166160383744, 0.7165486352017026, 0.7169912220471891, 0.7172535522530565, 0.7175222544477483, 0.7178873705483534, 0.7181393363678916, 0.7183686127652159, 0.7186460506786935, 0.7189807587167584, 0.7191723041320345, 0.7193283643847006, 0.7194327159364265, 0.7194347743032111, 0.7197137471524608, 0.7199458835054721, 0.720131201733596, 0.7203632592359286, 0.7204889902014532, 0.7206049051496688, 0.7207325474179661, 0.7208807079182155, 0.7209621161499263, 0.7211272717868583, 0.7212872966267239, 0.721410523069219, 0.7215622119498261, 0.7217131089229102, 0.7217831896511758, 0.721912313199927, 0.7220795488525008, 0.7222339387003531, 0.7223616470465899, 0.7224753880686742, 0.7226531521846496, 0.7227466363963596, 0.7228896154524439, 0.7

{'valid auc-mean': [0.6972921095094798, 0.7039204008308523, 0.707949006904624, 0.7090539519242421, 0.7104669847255273, 0.7114342553603127, 0.7125682226297781, 0.7132153819555341, 0.713609331967815, 0.7141828562889181, 0.7144764470987379, 0.7149857281140314, 0.7154178670999461, 0.7158497981558817, 0.7161573226198347, 0.7165045269661113, 0.7169678073343961, 0.7173047537718859, 0.7175473570189577, 0.7179718560033367, 0.7182528817130202, 0.7185537013944217, 0.7188152739725627, 0.7191247083112343, 0.7194154671948795, 0.7195353146342759, 0.7196691884663305, 0.7199659403056581, 0.7201310869135321, 0.7203770328532849, 0.7205237567108416, 0.7207384996552223, 0.7208391960599956, 0.720962583897883, 0.7210827352774126, 0.7211914386153021, 0.7213541913643988, 0.7214615955343818, 0.7216054968132495, 0.721597348183341, 0.7217371542870378, 0.7219181683691026, 0.7221134493260492, 0.7222302268593971, 0.7224306653704399, 0.7226042768346934, 0.7228107924054225, 0.722925058604171, 0.7229996534009157, 0.723

{'valid auc-mean': [0.6908354575146266, 0.7018420135597921, 0.7073818150116781, 0.7112760711121529, 0.7140154347019155, 0.7164601944343567, 0.7173870588689946, 0.7182099694869774, 0.7189678227974704, 0.71972126798855, 0.7200081106967597, 0.7204018121746731, 0.7205673687650535, 0.7207065597655394, 0.7212880018983295, 0.7213661635799754, 0.7213179219686032, 0.7214759879939067, 0.72165148257264, 0.7217955720262467, 0.7218237616173359, 0.7217537919081637, 0.7217682938578491, 0.7218706160706679, 0.7218844329603926, 0.7219433544413882, 0.7220267156064594, 0.7221804411985041, 0.7221831837934883, 0.7221656996735648, 0.7221029197979068, 0.7221143388288308, 0.7221458810335935, 0.7221982951484799], 'valid auc-stdv': [0.006434067162525354, 0.0036438895277781146, 0.004197040026085208, 0.005751412380544917, 0.004848408439488886, 0.004572357661158755, 0.004042219529667218, 0.004051664211201198, 0.003728587963109949, 0.003963368670142227, 0.0038096520272075467, 0.003920921042792551, 0.0037090575806062

{'valid auc-mean': [0.6988518590819139, 0.7054098182154651, 0.7089688716092043, 0.7110580151495577, 0.7130211689712582, 0.7139489429577562, 0.7153005660439735, 0.7158153291926675, 0.7164253287855255, 0.7170254363486223, 0.7173253050819257, 0.71776681253027, 0.7182972559361488, 0.7186564918876034, 0.718982100794765, 0.7192100688514865, 0.7193069956002437, 0.7194393551350841, 0.7196083956482213, 0.7199282077419662, 0.720140695871308, 0.7203563472376592, 0.7205615941362531, 0.7208312743290097, 0.7209737989679891, 0.7210538136672061, 0.7212016855896605, 0.721349107325381, 0.721423762741233, 0.7215720095914206, 0.7217111606123666, 0.7218590477438588, 0.7219992414265629, 0.722124016223978, 0.7222026450049008, 0.7223179099701739, 0.7223097064102292, 0.7223500828580703, 0.7224710685073856, 0.7225120418859946, 0.7226042572888175, 0.722685226690882, 0.7227491986142458, 0.7228703829514043, 0.7229540520107359, 0.7229560186300777, 0.7230451004538089, 0.7230714823799266, 0.7232076321802459, 0.723236

{'valid auc-mean': [0.6955591606321905, 0.7052729024838361, 0.7089763771618044, 0.7105832079159587, 0.7132632995991822, 0.7149923646800412, 0.7155842152699762, 0.7158965701049885, 0.7164857215974997, 0.7168093873593637, 0.7171015836185667, 0.7176131077175733, 0.7177641152436477, 0.7179639081022765, 0.7180569811568116, 0.7181048551684909, 0.7181694641168498, 0.7182802874409796, 0.7184954970685403, 0.7187834275978239, 0.7189020371696571, 0.719293354942297, 0.7192634486287784, 0.7193192375051441, 0.7192656214982455, 0.7194183574750462, 0.7194417633120459, 0.7194515046402206, 0.7194867245409476], 'valid auc-stdv': [0.0048316493473673835, 0.004891556319589137, 0.006987940629437897, 0.006681857305758517, 0.006916148987848341, 0.0065103733103477595, 0.006157453599571623, 0.005504260017649271, 0.005113887581092938, 0.004637976853593715, 0.004647756005759744, 0.004642671801495051, 0.004500565829128573, 0.0043971504377717414, 0.004368901019888278, 0.00413930705979925, 0.004223561358002831, 0.004

 **<font color='teal'> Print the best result by using the '.max' function.</font>**

In [19]:
lgbBO.max

{'target': 0.0,
 'params': {'lambda_l1': 0.041562969647767165,
  'lambda_l2': 0.0007266741731021487,
  'max_depth': 31.954553057985578,
  'min_child_samples': 7221.74398341644,
  'min_data_in_leaf': 733.9763881811416,
  'num_leaves': 1697.9801849557773}}

Review the process at each step by using the '.res[0]' function.

In [20]:
lgbBO.res[0]

{'target': 0.0,
 'params': {'lambda_l1': 0.041562969647767165,
  'lambda_l2': 0.0007266741731021487,
  'max_depth': 31.954553057985578,
  'min_child_samples': 7221.74398341644,
  'min_data_in_leaf': 733.9763881811416,
  'num_leaves': 1697.9801849557773}}